# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [57]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

In [4]:
db_filename = 'DisasterResponse.db'
db_tablename = 'messages'

In [10]:
labels = [ 
 'related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report'
]

In [9]:
# load data from database
engine = create_engine(f'sqlite:///{db_filename}')
df = pd.read_sql_table(db_tablename, engine)
X = df.message
Y = df[labels]

### 2. Write a tokenization function to process your text data

In [35]:
import re

url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
url_placeholder = '_URL_'

lemmatizer = WordNetLemmatizer()

def create_placeholder_for_urls(text):
    return re.sub(url_regex, url_placeholder, text)

def tokenize(text):
    lower_text = text.lower().strip()
    placeholder_text = create_placeholder_for_urls(lower_text)
    tokens = word_tokenize(placeholder_text)
    return list(map(lemmatizer.lemmatize, tokens))

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [52]:
pipeline = Pipeline([
        ('vectorizer', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('classifier', MultiOutputClassifier(RandomForestClassifier(n_estimators = 10)))
])

In [ ]:
pipeline.fit(X_train, y_train)

In [82]:
y_pred = pipeline.predict(X_test)

In [126]:
def display_results(column_name, column_index, y_test, y_pred):
    
    curr_y_test = y_test[[column_name]]
    curr_y_pred = y_pred[:, column_index]
    
    labels = [0,1] # np.unique(curr_y_test)
    confusion_mat = confusion_matrix(curr_y_test, curr_y_pred, labels=labels)
    accuracy = (curr_y_test[column_name] == curr_y_pred).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

In [127]:
for idx, name in enumerate(list(y_test.columns)):
    
    print(f'Category: {name} at index {idx}')
    
    display_results(name, idx, y_test, y_pred)
    
    print('\n\n')

Category: related at index 0
Labels: [0, 1]
Confusion Matrix:
 [[ 518 1026]
 [ 355 4607]]
Accuracy: 0.7832263978001833



Category: request at index 1
Labels: [0, 1]
Confusion Matrix:
 [[5392   87]
 [ 664  403]]
Accuracy: 0.8852734494347694



Category: offer at index 2
Labels: [0, 1]
Confusion Matrix:
 [[6517    0]
 [  29    0]]
Accuracy: 0.9955698136266422



Category: aid_related at index 3
Labels: [0, 1]
Confusion Matrix:
 [[3373  479]
 [1292 1402]]
Accuracy: 0.7294531011304614



Category: medical_help at index 4
Labels: [0, 1]
Confusion Matrix:
 [[5973   27]
 [ 515   31]]
Accuracy: 0.9172013443324167



Category: medical_products at index 5
Labels: [0, 1]
Confusion Matrix:
 [[6208   18]
 [ 282   38]]
Accuracy: 0.9541704857928506



Category: search_and_rescue at index 6
Labels: [0, 1]
Confusion Matrix:
 [[6333    5]
 [ 203    5]]
Accuracy: 0.9682248701497097



Category: security at index 7
Labels: [0, 1]
Confusion Matrix:
 [[6428    2]
 [ 116    0]]
Accuracy: 0.9819737244118546


In [118]:
display_results(y_test, y_pred)

Labels: [0, 1]
Confusion Matrix:
 [[5392   87]
 [ 664  403]]
Accuracy: 0.8852734494347694


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.